In [3]:
import pandas as pd
import glob
import os
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

# 설정 변수
random_state = 12
mlrsnet_root = '/home/son/ml/datasets/MLRSNet'      # MLRSNet 데이터셋의 루트 디렉토리 위치
output_dir = '/home/son/ml/xai/split_data'   # CSV 파일 저장 경로 설정

labels_dir = os.path.join(mlrsnet_root, 'Labels')

label_files = glob.glob(os.path.join(labels_dir, '*.csv'))

all_labels_df = pd.DataFrame()

for file in label_files:
    df = pd.read_csv(file)
    class_name = os.path.splitext(os.path.basename(file))[0]
    df['image'] = df['image'].apply(lambda x: os.path.join('Images', class_name, x))
    all_labels_df = pd.concat([all_labels_df, df], ignore_index=True)

# 이미지 경로에 MLRSNet 루트 경로를 추가
all_labels_df['image'] = all_labels_df['image'].apply(lambda x: os.path.join(mlrsnet_root, x))

# 레이블 데이터 준비
X = all_labels_df['image'].values
y = all_labels_df.iloc[:, 1:].values  # 레이블 컬럼들

# 데이터 분할 비율 설정 (예: 40% train, 10% val, 50% test)
train_ratio = 0.2
val_ratio = 0.1
test_ratio = 0.7

# 첫 번째 분할: Train과 Temp(Val+Test)
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=(1 - train_ratio), random_state=random_state)
for train_index, temp_index in msss.split(X, y):
    X_train, X_temp = X[train_index], X[temp_index]
    y_train, y_temp = y[train_index], y[temp_index]

# 두 번째 분할: Val과 Test
val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)  # Temp 데이터에서의 비율
msss = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=random_state)
for val_index, test_index in msss.split(X_temp, y_temp):
    X_val, X_test = X_temp[val_index], X_temp[test_index]
    y_val, y_test = y_temp[val_index], y_temp[test_index]

# 데이터프레임 재구성
train_df = pd.DataFrame(X_train, columns=['image'])
train_labels_df = pd.DataFrame(y_train, columns=all_labels_df.columns[1:])
train_df = pd.concat([train_df, train_labels_df], axis=1)

val_df = pd.DataFrame(X_val, columns=['image'])
val_labels_df = pd.DataFrame(y_val, columns=all_labels_df.columns[1:])
val_df = pd.concat([val_df, val_labels_df], axis=1)

test_df = pd.DataFrame(X_test, columns=['image'])
test_labels_df = pd.DataFrame(y_test, columns=all_labels_df.columns[1:])
test_df = pd.concat([test_df, test_labels_df], axis=1)

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

# CSV 파일로 저장
train_df.to_csv(os.path.join(output_dir, 'train.csv'), index=False)
val_df.to_csv(os.path.join(output_dir, 'val.csv'), index=False)
test_df.to_csv(os.path.join(output_dir, 'test.csv'), index=False)


Train set size: 21820
Validation set size: 10875
Test set size: 76466
